In [1]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15984 sha256=dbb575e91181e7d76db7fc0ae6cb8280d0e09e556ea147645afed7f8ed917e85
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [2]:
!ngrok authtoken 231aDBrawch35gOZZoCIMc5Z8Lb_5nmGDpG4oTEUY1ich83VN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
import pickle
from os import name
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, redirect, url_for
import datetime
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

###Conversion of information received as input to a dataframe



In [7]:
import datetime
from datetime import datetime
def Create_dataframe_for_prediction(data):
  
  # Date
  y = data[0][0:4]
  m = data[0][5:7]
  d = data[0][8:10]
  Date = d+'/'+m+'/'+y
  Date

  date = datetime.strptime(Date, "%d/%m/%Y")
  date

  data[0]=date

  # Hour
  if (int(data[1][0])==0): 
    data[1] = int(data[1][1]) 
  else:
    data[1] = int(data[1][0] + data[1][1])
  hour = data[1]

  holiday = 1 if data[2]=="yes" else 0

  functioning_day = 1 if data[3]=="yes" else 0

  temperature = float(data[4])

  humidity = int(data[5])

  windspeed = float(data[6])

  visibility = int(data[7])

  dew_point_temperature = float(data[8])

  solar_radiation = float(data[9])

  rainfall = float(data[10])

  snowfall = float(data[11])

  number_month = date.month


  # To determine the season
  season=''
  # We get the current day of the year
  day_of_the_year = date.timetuple().tm_yday

  spring = range(80, 172)
  summer = range(172, 264)
  autumn = range(264, 355)
  # winter = everything else

  if day_of_the_year in spring:
    season = 'Spring'
  elif day_of_the_year in summer:
    season = 'Summer'
  elif day_of_the_year in autumn:
    season = 'Autumn'
  else:
    season = 'Winter'

  season_winter = 1 if season=="Winter" else 0
  season_spring = 1 if season=="Spring" else 0
  season_summer = 1 if season=="Summer" else 0
  season_autumn = 1 if season=="Autumn" else 0


  # Part of the day : Morning, Afternoon, Evening, Night
  Part_of_the_day = part_of_the_day(int(hour))

  part_of_the_day_morning = 1 if Part_of_the_day=="Morning" else 0
  part_of_the_day_afternoon = 1 if Part_of_the_day=="Afternoon" else 0
  part_of_the_day_evening = 1 if Part_of_the_day=="Evening" else 0
  part_of_the_day_night = 1 if Part_of_the_day=="Night" else 0


  # Week-end day or not
  w_d = date.weekday()
  day=""
  if(w_d==0):
    day="Monday"
  elif(w_d==1):
    day="Tuesday"
  elif(w_d==2):
    day="Wednesday"
  elif(w_d==3):
    day="Thursday"
  elif(w_d==4):
    day="Friday"
  elif(w_d==5):
    day="Saturday"
  elif(w_d==6):
    day="Sunday"

  week_end_day_yes = 1 if (day=="Saturday" or day=="Sunday") else 0
  week_end_day_no = 0 if (day=="Saturday" or day=="Sunday") else 1

  day_week_monday = 1 if day=="Monday" else 0
  day_week_tuesday = 1 if day=="Tuesday" else 0
  day_week_wednesday = 1 if day=="Wednesday" else 0
  day_week_thursday = 1 if day=="Thursday" else 0
  day_week_friday = 1 if day=="Friday" else 0
  day_week_saturday = 1 if day=="Saturday" else 0
  day_week_sunday = 1 if day=="Sunday" else 0

 
  
  columns = [date, hour, temperature, humidity, windspeed, visibility, dew_point_temperature, solar_radiation, rainfall, snowfall, holiday, functioning_day, 
             number_month, season_autumn, season_spring, season_summer, season_winter, day_week_friday, day_week_monday, day_week_saturday, day_week_sunday, 
             day_week_thursday, day_week_tuesday, day_week_wednesday, part_of_the_day_afternoon, part_of_the_day_evening, part_of_the_day_morning, part_of_the_day_night, 
             week_end_day_no, week_end_day_yes]
  
  column_names = ["Date", "Hour", "Temp", "Hum", "Wind", "Vis", "Dew", "Solar", "Rain", "Snow", "Holiday", "FDay", 
                  "Number_month", "Seasons_Autumn", "Seasons_Spring", "Seasons_Summer", "Seasons_Winter", "Day_week_Friday", "Day_week_Monday", "Day_week_Saturday", "Day_week_Sunday", 
                  "Day_week_Thursday", "Day_week_Tuesday", "Day_week_Wednesday", "Part_of_the_day_Afternoon", "Part_of_the_day_Evening", "Part_of_the_day_Morning", "Part_of_the_day_Night", 
                  "Week_end_day_no", "Week_end_day_yes"]

  df_prediction = pd.DataFrame([columns], columns=column_names)
  df_prediction['Date'] = pd.to_numeric(pd.to_datetime(df_prediction['Date']))

  df_complete = pd.concat([df_bike2, df_prediction], ignore_index = True)

  df_complete.drop(["Bike_Count"], axis = 1, inplace = True)


  X = df_complete
  scaler = StandardScaler()
  scaler.fit(X)
  X_scaled = scaler.transform(X)

  df_pred = np.array(list(X_scaled[8760]), ndmin=2)

  return df_pred


####App

In [9]:
app = Flask(__name__, template_folder='/content', static_folder='/content/static')

run_with_ngrok(app)

model_grad_boosting_reg = pickle.load(open('/content/model_grad_boosting_reg.pkl', 'rb'))
df_bike2 = pickle.load(open('/content/df_bike2.pkl', 'rb'))

def part_of_the_day(hour):
    if(hour < 6):
        return "Night"
    elif (hour < 12):
        return "Morning"
    elif (hour < 18):
        return "Afternoon"
    else :
          return "Evening"

@app.route('/')
def home():
    return render_template("index_accueil.html")

@app.route('/get_prediction', methods=['POST'])
def get_prediction():    

    input = [x for x in request.form.values()]
    input.append("")

    df_prediction = Create_dataframe_for_prediction(input)

    prediction = model_grad_boosting_reg.predict(df_prediction)

    return render_template("index_accueil.html", output='Predicted Bike Count : {}'.format(prediction))

app.run()

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fafe-35-186-189-191.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Jan/2022 11:30:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 11:30:50] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 11:30:50] "GET /static/bike.jpg HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2022 11:30:51] "GET /favicon.ico HTTP/1.1" 404 -


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
